In [677]:
import sys
import random
import time

from montgomery_xz import KummerLine
from montgomery_isogeny import KummerLineIsogenyComposite
from utilities import torsion_basis

In [1]:
lam = 128
ternary = True
sk_choices = [1, 2]
params_binSIDH = [95, 150, 197]
params_terSIDH = [67, 105, 139]
params_t1bin = [35, 45, 60]
params_t1ter = [30, 35, 50]
params = params_binSIDH
t = 95
t1 = 35
#t = 10
#t1 = 6

In [2]:
t1

35

In [3]:
def make_prime(p):
    '''
    Given a value `p`, finds a cofactor `f`
    such that p*f - 1 is prime.
    '''
    for i in range(1000):
        if (p*i - 1).is_prime():
            return p*i - 1, i

In [4]:
A = 2*prod(Primes()[:2*t:2])*prod(Primes()[:2*t1:2]) # The 2* ensures that p \equiv 3 mod 4
B = prod(Primes()[1:2*t:2])*prod(Primes()[1:2*t1:2])
p, f = make_prime(A*B)

In [5]:
factor(A)

2^3 * 5^2 * 11^2 * 17^2 * 23^2 * 31^2 * 41^2 * 47^2 * 59^2 * 67^2 * 73^2 * 83^2 * 97^2 * 103^2 * 109^2 * 127^2 * 137^2 * 149^2 * 157^2 * 167^2 * 179^2 * 191^2 * 197^2 * 211^2 * 227^2 * 233^2 * 241^2 * 257^2 * 269^2 * 277^2 * 283^2 * 307^2 * 313^2 * 331^2 * 347^2 * 353 * 367 * 379 * 389 * 401 * 419 * 431 * 439 * 449 * 461 * 467 * 487 * 499 * 509 * 523 * 547 * 563 * 571 * 587 * 599 * 607 * 617 * 631 * 643 * 653 * 661 * 677 * 691 * 709 * 727 * 739 * 751 * 761 * 773 * 797 * 811 * 823 * 829 * 853 * 859 * 877 * 883 * 907 * 919 * 937 * 947 * 967 * 977 * 991 * 1009 * 1019 * 1031 * 1039 * 1051 * 1063 * 1087 * 1093 * 1103 * 1117 * 1129

In [6]:
factor(B)

3^2 * 7^2 * 13^2 * 19^2 * 29^2 * 37^2 * 43^2 * 53^2 * 61^2 * 71^2 * 79^2 * 89^2 * 101^2 * 107^2 * 113^2 * 131^2 * 139^2 * 151^2 * 163^2 * 173^2 * 181^2 * 193^2 * 199^2 * 223^2 * 229^2 * 239^2 * 251^2 * 263^2 * 271^2 * 281^2 * 293^2 * 311^2 * 317^2 * 337^2 * 349^2 * 359 * 373 * 383 * 397 * 409 * 421 * 433 * 443 * 457 * 463 * 479 * 491 * 503 * 521 * 541 * 557 * 569 * 577 * 593 * 601 * 613 * 619 * 641 * 647 * 659 * 673 * 683 * 701 * 719 * 733 * 743 * 757 * 769 * 787 * 809 * 821 * 827 * 839 * 857 * 863 * 881 * 887 * 911 * 929 * 941 * 953 * 971 * 983 * 997 * 1013 * 1021 * 1033 * 1049 * 1061 * 1069 * 1091 * 1097 * 1109 * 1123 * 1151

In [7]:
f

24

In [727]:
ceil(log(p,2))

4741

In [ ]:
FF.<x> = GF(p)[]
F.<i> = GF(p^2, modulus=x^2+1) 
E0 = EllipticCurve(F, [0, 6, 0, 1, 0])
E0.set_order((p+1)**2) 

In [ ]:
PA, QA = torsion_basis(E0, A/2)
PB, QB = torsion_basis(E0, B)

In [ ]:
Pa, Qa = torsion_basis(E0,A/2)

In [ ]:
if A//2 * PA == E0(0, 0):
    PA = PA + QA
elif A//2 * QA == E0(0, 0):
    QA = PA + QA 

In [732]:
assert A//2 * PA != E0(0, 0) and A//2 * QA != E0(0, 0)

In [733]:
E0 = KummerLine(E0)
xPA, xQA = E0(PA[0]), E0(QA[0])
xPB, xQB = E0(PB[0]), E0(QB[0])

In [693]:
def secret_generate(t, t1):
    """
    Given t, generate secret key of Alice and Bob,
    The length of secret key is t
    for example = -1,2,2,1,-2.....
    -1 means G_2 
    p = fAB -1 
    A = p1^2...pt1^2...pt
    """
    choices_1 = [-1, 1, 2, -2]
    #choices_1 = [-1, 1, 2, -2]
    sk1 = random.choices(choices_1, k = t1)
    choices = [-1, 1]
    sk2 = random.choices(choices, k = t - t1)
    return sk1 + sk2

In [532]:
def compute_kernel_scalars(s, t ,t1, Alice=True):
    """ 
    Given a ternary secret `s`, returns scalars `B0` and `B1`
    such that the isogeny associated with `s` and orientation (P, Q)
    has kernel <[B0]*P + [B1]*Q>.
    The function also returns `order0` and `order1`, the orders
    of points [B0]*P and [B1]*Q, which is used in the isogeny computations.
    """
    B0 = 1
    B1 = 1
    order0 = 1
    order1 = 1


    if Alice:
        P = Primes()[:2*t:2]
    else:
        P = Primes()[1:2*t:2]

    for i, p in enumerate(P):
            
        if i < t1 and s[i] == 2:
            B0 *= p**2
            order1 *= p**2
        elif i < t1 and s[i] == 1:
            B0 *= p**2
            B1 *= p
            order1 *= p
        elif i < t1 and s[i] == -1:
            B1 *= p**2
            B0 *= p
            order0 *= p
        elif i < t1 and s[i] == -2:
            B1 *= p**2
            order0 *= p**2
        elif i < t1 and s[i] == 0:
            B0 *= p**2
            B1 *= p**2
        elif i >= t1 and s[i] == 1:
            B0 *= p
            order1 *= p
        elif i >= t1 and s[i] == -1:
            B1 *= p
            order0 *= p
        elif s[i]== 0:
            B0 *= p
            B1 *= p            
    return B0, B1, order0, order1

In [613]:
def keygen(A_points, B_points, Alice=True):
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points

    # Generate the secret data
    sk = secret_generate(t, t1)
    print(sk)
    B0, B1, order0, order1 = compute_kernel_scalars(sk, t, t1 ,Alice=Alice)
    sk = (B0, B1, order0, order1)
    
    # Compute the isogeny kernels
    xK0 = xP * B0
    xK1 = xQ * B1

    # Compute the first isogeny
    phi0 = KummerLineIsogenyComposite(E0, xK0, order0)
    xK1 = phi0(xK1)

    # Evaluate action on aux data
    xR, xS = phi0(xR), phi0(xS)

    # Compute the second isogeny from the codomain of phi0
    E1 = phi0.codomain()
    phi1 = KummerLineIsogenyComposite(E1, xK1, order1)

    # Evaluate action on aux data
    xR, xS = phi1(xR), phi1(xS)

    # Generate the masking values
    modulus = B if Alice else A # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E1 = phi1.codomain()
    pk = (E1, xR, xS)

    return sk, pk

In [611]:
"""
def keygen(A_points, B_points, Alice=True):
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points

    # Generate the secret data
    sk = secret_generate(t, t1)
    print(sk)
    B0, B1, order0, order1 = compute_kernel_scalars(sk, t, t1 ,Alice=Alice)
    sk = (B0, B1, order0, order1)
    
    # Compute the isogeny kernels
    xK0 = xP * B0
    xK1 = xQ * B1

    ker = xK0 + xK1
    xker = E0(ker[0])

    # Compute the second isogeny from the codomain of phi0
    phi1 = KummerLineIsogenyComposite(E0, xker, order1*order0)

    
    xR = E0(xR[0])
    xS = E0(xS[0])
    # Evaluate action on aux data
    xR, xS = phi1(xR), phi1(xS)

    # Generate the masking values
    modulus = B if Alice else A # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E1 = phi1.codomain()
    pk = (E1, xR, xS)

    return sk, pk
"""


In [534]:
def shared(sk, pk):
    # Parse the private/public keys
    B0, B1, order0, order1 = skA
    E, xR, xS = pkB
    
    # Compute the isogeny kernels
    xK0 = xR * B0
    xK1 = xS * B1

    # Compute the first isogeny
    phi0 = KummerLineIsogenyComposite(E, xK0, order0)
    xK1 = phi0(xK1)

    # Compute the second isogeny from the codomain of phi0
    E = phi0.codomain()
    phi1 = KummerLineIsogenyComposite(E, xK1, order1)

    EAB = phi1.codomain()
    return EAB.j_invariant()

In [ ]:
N = 1
tt = [0, 0, 0, 0]

for _ in range(N):
    t0 = time.process_time_ns()
    skA, pkA = keygen((xPA, xQA), (xPB, xQB), Alice=True)
    tt[0] += time.process_time_ns() - t0

    t0 = time.process_time_ns()
    skB, pkB = keygen((xPB, xQB), (xPA, xQA), Alice=False)
    tt[1] += time.process_time_ns() - t0

    t0 = time.process_time_ns()
    ssA = shared(skA, pkB)
    tt[2] += time.process_time_ns() - t0

    t0 = time.process_time_ns()
    ssB = shared(skB, pkA)
    tt[3] += time.process_time_ns() - t0

    assert ssA == ssB

tt = [float(t) / N / 10^6 for t in tt]

print(f"KeyGen_A took {(tt[0]):.1f} ms")
print(f"KeyGen_B took {(tt[1]):.1f} ms")
print(f"shared_A took {(tt[2]):.1f} ms")
print(f"shared_B took {(tt[3]):.1f} ms")

[2, 1, 2, 2, -2, -2, 2, 1, 1, 2, 1, -2, 1, -2, 2, 2, -1, 2, 2, -2, -1, 2, -2, -1, 2, 1, -1, -2, -2, -1, 1, 2, 2, -2, 2, -2, -2, -1, -1, 1, -1, -2, 1, -2, -1, 1, -2, -1, 2, 1, 1, 2, 2, 1, 2, -1, 1, -1, 2, -2, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1]
